In [1]:
using Rocket
using MacroTools
using ReactiveMP
using GraphPPL
using BenchmarkTools
using Distributions

┌ Info: Precompiling ReactiveMP [a194aa59-28ba-4574-a09c-4a745416d6e3]
└ @ Base loading.jl:1278
┌ Warning: Package ReactiveMP does not have Rocket in its dependencies:
│ - If you have ReactiveMP checked out for development and have
│   added Rocket as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with ReactiveMP
└ Loading Rocket into ReactiveMP from project dependency, future warnings for ReactiveMP are suppressed.
┌ Info: Precompiling GraphPPL [b3f8163a-e979-4e85-b43e-1f63d8c8b42c]
└ @ Base loading.jl:1278
┌ Warning: Package GraphPPL does not have ReactiveMP in its dependencies:
│ - If you have GraphPPL checked out for development and have
│   added ReactiveMP as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with GraphPPL
└ Loading ReactiveMP into GraphPPL from project dependency, future 

In [77]:
@model [ default_factorisation = MeanField() ] function coin_model()
    a = datavar(Float64)
    b = datavar(Float64)
    y = datavar(Float64)
    
    θ ~ Beta(a, b)
    y ~ Bernoulli(θ)
    
    return y, a, b, θ
end

coin_model (generic function with 1 method)

In [81]:
model, _ = coin_model(options = (default_factorisation = FullFactorisation(), ));

In [82]:
getnodes(model)

2-element Array{FactorNode,1}:
 FactorNode:
 form            : Beta
 sdtype          : Stochastic()
 interfaces      : (Interface(out), Interface(a), Interface(b))
 factorisation   : ((1, 2, 3),)
 local marginals : (:out_a_b,)
 metadata        : nothing

 FactorNode:
 form            : Bernoulli
 sdtype          : Stochastic()
 interfaces      : (Interface(out), Interface(p))
 factorisation   : ((1, 2),)
 local marginals : (:out_p,)
 metadata        : nothing


In [11]:
model_options(NamedTuple{()}(()))

ModelOptions{DefaultMessageGate,DiscontinueStreamTransformer,FullFactorisation}(DefaultMessageGate(), DiscontinueStreamTransformer(), FullFactorisation())

In [26]:
using MacroTools

In [29]:
@capture(:(a[1]), a_Symbol)

false

In [1]:
import MacroTools: @capture

In [7]:
@capture(:(a), s_Symbol)

LoadError: UndefVarError: MacroTools not defined